In [10]:
import h5py
import sys
import os

import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d

# Find the repo root (parent of current folder)
repo_root = os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), '..')) if '__file__' not in globals() else os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))
sys.path.append(repo_root)

# Add only immediate subfolders of the repo root to sys.path (much faster)
for item in os.listdir(repo_root):
    item_path = os.path.join(repo_root, item)
    if os.path.isdir(item_path):
        sys.path.append(item_path)

from read_scope_data import read_hdf5_scope_data, read_hdf5_scope_tarr, read_hdf5_all_scopes_channels, read_scope_channel_descriptions
from read_hdf5 import print_hdf5_info, print_scope_channel_descriptions
from process_xray_bdot_hdf5 import get_xray_data, process_shot_xray

%matplotlib qt

In [9]:
ifn = r"F:\AUG2025\P24\10_He1kG430G_5800A_K-25_2025-08-12.hdf5"
# print_hdf5_info(ifn)

In [13]:
shot_num = 1
with h5py.File(ifn, 'r') as f:
    result = read_hdf5_all_scopes_channels(f, shot_num)
    tarr_x, xray_data = get_xray_data(result)
    threshold = [5, 60]
    min_ts = 0.8e-6
    d = 0.1
    pulse_tarr, pulse_amp = process_shot_xray(tarr_x, xray_data, min_ts, d, threshold, debug=True)

Applying Savitzky-Golay filter...
Analyzing optimal downsample rate...

Analyzing downsample rates (minimum 2.0e+02 samples needed for 8.0e-04 ms features):
Rate  2: Mean error = 3.53e-05, Samples in min_timescale = 100.0
Rate  5: Mean error = 1.33e-04, Samples in min_timescale = 40.0
Rate 10: Mean error = 3.20e-04, Samples in min_timescale = 20.0
Rate 20: Mean error = 7.25e-04, Samples in min_timescale = 10.0
Rate 50: Mean error = 1.26e-03, Samples in min_timescale = 4.0

Recommended downsample rate: 2
Downsample rate: 2
Computing baseline...
Computing thresholds...
Detecting pulses...


In [19]:
with h5py.File(ifn, 'r') as f:
    cine_name_arr = f['Control/FastCam/cine file name'][()]
    print(cine_name_arr[0][:2])

b'00'


In [7]:
shot_num = 1
with h5py.File(ifn, 'r') as f:
    cine_narr = f['Control/FastCam/cine file name'][()]
    cam_file = cine_narr[shot_num-1]
    if shot_num != int(cam_file[-8:-5]):
        print(f"Warning: shot number {shot_num} does not match {cam_file}")
    print(cam_file.decode('utf-8'))

00_He1kG430G_5800A_K-25_shot001.cine


In [ ]:
scope_name = 'xrayscope'
channel_name = 'C2'
shot_number = 1
data = read_hdf5_scope_data(ifn, scope_name, channel_name, shot_number)
tarr = read_hdf5_scope_tarr(ifn, scope_name)

In [ ]:
# Test the updated process_shot_bdot function
from process_xray_bdot_hdf5 import process_shot_bdot

# Test the function with the HDF5 file
stft_time, freq, stft_matrix1, stft_matrix2, stft_matrix3, channel_info = process_shot_bdot(ifn, shot_number=1, debug=True)

# Print channel information
print("\n=== Channel Information ===")
for channel, info in channel_info.items():
    field_comp = info['field_component']
    position = info['position']
    desc = info['description']
    print(f"{channel}: {field_comp} at {position} - {desc}")

# Check if data was successfully read
print(f"\nSTFT matrices availability:")
print(f"Matrix 1 (C1): {'Available' if stft_matrix1 is not None else 'None'}")
print(f"Matrix 2 (C2): {'Available' if stft_matrix2 is not None else 'None'}")
print(f"Matrix 3 (C3): {'Available' if stft_matrix3 is not None else 'None'}")

if stft_time is not None:
    print(f"STFT time array shape: {stft_time.shape}")
    print(f"Frequency array shape: {freq.shape}")